# Load & Structure FDR Data

into normalized table
taken from 

In [43]:
# import modules

import numpy as np
import pandas as pd

In [2]:
# set constants

csv_filename = "570991.csv"

In [3]:
# Read first 10 rows of CSV file, with no header row

df_raw = pd.read_csv(csv_filename, header=None, nrows=10)
df_raw.head(10)

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,Vehicle ID/Registration: N121JM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Source of Data: Gulfstream GIV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Date of Event: 06/03/14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Location: Bedford,Massachusetts,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Time,Left Main Squat Switch,Right Main Squat Switch,Nose Squat Switch,Calibrated airspeed,Lateral acceleration,Longitudinal acceleration,Vertical acceleration,Pitch angle,Roll angle,...,Right thrust reverser deployed,Right thrust reverser stow/transit,Right tur vib ind HP,Right turbine gas temp,Right wheel brake pressure,Static Air Temp,Yaw damper status,Auto-pilot status,A/T Hold,A/T Engaged
5,(SRN),(),(),(),(kts),(g),(g),(g),(deg),(deg),...,(),(),(IN/SEC),(degC),(PSI),(degC),(),(),(),()
6,NaN,"%N(0.0:0.0=""Air"",1.0:1.0=""Ground"")","%N(0.0:0.0=""Air"",1.0:1.0=""Ground"")","%N(0.0:0.0=""Air"",1.0:1.0=""Ground"")",NUMBER,NUMBER,NUMBER,NUMBER,NUMBER,NUMBER,...,"%N(0.0:0.0=""Deploy"",1.0:1.0=""-"")","%N(0.0:0.0=""In transit"",1.0:1.0=""Stow"")",NUMBER,NUMBER,NUMBER,NUMBER,"%N(0.0:0.0=""-"",1.0:1.0=""ENG"")","%N(0.0:0.0=""-"",1.0:4.0=""ENG"")","%N(0:0=""-"",1:1=""Hold"")","%N(0.0:0.0=""-"",1.0:1.0=""ENG"")"
7,143488.0156,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,143488.0469,NaN,NaN,NaN,NaN,NaN,NaN,1.019,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,143488.0625,NaN,NaN,NaN,NaN,NaN,-0.017,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# NOTE: First 4 rows are overview info for NTSB crash Investigation, so grab
############## TBC - missing state in second column for Location

fdr_overview = df_raw.iloc[0:4,0].values.tolist()
fdr_overview

['Vehicle ID/Registration: N121JM',
 'Source of Data: Gulfstream GIV',
 'Date of Event: 06/03/14',
 'Location: Bedford']

In [5]:
# NOTE: There are 58 columns, whose names are in fifth row, so grab

fdr_columns = df_raw.loc[4, :].values.tolist()
fdr_columns[:5]

['Time',
 'Left Main Squat Switch',
 'Right Main Squat Switch',
 'Nose Squat Switch',
 'Calibrated airspeed']

In [6]:
# NOTE: The next two rows contain metadata about the columns, so grad
#############  TDC - use dict for column names and metadata

fdr_metadata1 = df_raw.loc[5, :].values.tolist()
print(fdr_metadata1[:5])

fdr_metadata2 = df_raw.loc[6, :].values.tolist()
print(fdr_metadata2[:5])

['(SRN)', '()', '()', '()', '(kts)']
[nan, '%N(0.0:0.0="Air",1.0:1.0="Ground")', '%N(0.0:0.0="Air",1.0:1.0="Ground")', '%N(0.0:0.0="Air",1.0:1.0="Ground")', 'NUMBER']


In [7]:
# Read entire FDR file
# ...starting with real data in the eighth row and column names given, assume str dtype

# df_raw = pd.read_csv(csv_filename, skiprows=6, header=0, names=fdr_columns, dtype=str)
df_raw = pd.read_csv(csv_filename, skiprows=6, header=0, names=fdr_columns)
#print(df_raw.head())
#print('------')
print(df_raw.tail())

               Time Left Main Squat Switch Right Main Squat Switch  \
191995  147211.5313                    NaN                     NaN   
191996  147211.5469                    NaN                     NaN   
191997  147211.5625                    NaN                     NaN   
191998  147211.5781                    NaN                     NaN   
191999  147211.5938                    NaN                     NaN   

       Nose Squat Switch  Calibrated airspeed  Lateral acceleration  \
191995               NaN                  NaN                   NaN   
191996               NaN                  NaN                   NaN   
191997               NaN                  NaN                   NaN   
191998               NaN                  NaN                -0.037   
191999               NaN                  NaN                   NaN   

        Longitudinal acceleration  Vertical acceleration  Pitch angle  \
191995                        NaN                    NaN          NaN   
191996

In [8]:
# NOTE info about the FDR df

df_raw.info(max_cols=0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192000 entries, 0 to 191999
Columns: 58 entries, Time to A/T Engaged
dtypes: float64(35), object(23)
memory usage: 85.0+ MB


In [9]:
# NOTE: how sparse this df is

nonNaN_count = sum(df_raw.count())
print("count of non-NaN values = ", nonNaN_count)
print("total cells in df = ", df_raw.size)
print("fraction of non-NaN values = ", (nonNaN_count/df_raw.size))
print("average values per row = ", (nonNaN_count/len(df_raw)))

count of non-NaN values =  452891
total cells in df =  11136000
fraction of non-NaN values =  0.0406690912356
average values per row =  2.35880729167


In [10]:
# Examine the cardinality and unique values of each column

print('{0}'.format("         column name    #col   #cnt #unique     first values"))

col_count = df_raw.count().tolist()
for i, (column, series) in enumerate(df_raw.iloc[:, :].iteritems()):
    nvalues = col_count[i]
    uvalues = list(set(series.dropna()))
    print('{0:>25.25s} {1:2d} {2:6d} {3:6d} {4}'.format(column, i, nvalues, len(uvalues), uvalues[:3]))


         column name    #col   #cnt #unique     first values
                     Time  0 192000 192000 [146160.07810000001, 146160.70310000001, 146160.82810000001]
   Left Main Squat Switch  1   3723      2 ['Air', 'Ground']
  Right Main Squat Switch  2   3724      2 ['Air', 'Ground']
        Nose Squat Switch  3   3724      2 ['Air', 'Ground']
      Calibrated airspeed  4   3724    457 [0.0, 0.5, 2.5]
     Lateral acceleration  5  14895     78 [-0.057999999999999996, -0.062, -0.001]
Longitudinal acceleration  6  14895    272 [0.252, 0.26000000000000001, 0.26600000000000001]
    Vertical acceleration  7  29789    198 [0.9890000000000001, 1.0170000000000001, 1.0190000000000001]
              Pitch angle  8  14894     99 [-0.69999999999999996, -0.5, 0.0]
               Roll angle  9   7447     70 [0.40000000000000002, 0.0, -0.40000000000000002]
         Magnetic heading 10   3724    456 [4.5700000000000003, 5.9800000000000004, 4.2199999999999998]
         True track angle 11   3723    9

In [64]:
# Convert the raw df format to a 'normalize' format of (stime, etime, variable, value)

# initialize the new normalized df with its five column names
norm_colnames = ['colno', 'stime', 'etime', 'value', 'delrows', 'colname']
norm_df = pd.DataFrame(columns=norm_colnames)
norm_df.colno = norm_df.colno.astype(int)
norm_df.delrows = norm_df.delrows.astype(int)

# grab the orginial column names from the raw df
raw_colnames = df_raw.columns

time_zero = df_raw.iloc[0,0]                     # note beginning time of FDR data
colval_dict = {}                                 # initialize dict for column values

for row_no, (row, series) in enumerate(df_raw.iloc[:, :].iterrows()):  # scan all variables (except time)
    print(row_no, 100*row_no/len(df_raw), end='\r')          # log progress
    
    new_time = series[0] - time_zero                 # note time of new raw row, minus beginning time
    for col_no, new_value in enumerate(series[1:]):  # iter columns, ignoring time in first column
        
        # bump col_no by one to account for time column
        col_no = col_no + 1
        
        if pd.notnull(new_value):                    # ignore null cells
            # are there prior readings for this column?
            if not col_no in colval_dict:      # first one!
                # no readings => add new column with (time, value, count of deleted rows)
                colval_dict[col_no] = (new_time, new_value, 0)
            else:
                # prior reading => check if old value == new value
                (old_time, old_value, old_delrows) = colval_dict[col_no]
                if old_value == new_value:
                    # same value => bump delrows counter & continue
                    colval_dict[col_no] = (old_time, old_value, old_delrows+1)
                else:
                    # different values => create new norm row to close previous values
                    norm_colno = col_no
                    norm_stime = old_time                  # inclusive of this time
                    norm_etime = new_time                  # exclusive of this time
                    norm_value = old_value
                    norm_delrows = old_delrows
                    norm_colname = raw_colnames[col_no]
                    norm_row = [norm_colno, norm_stime, norm_etime, norm_value, norm_delrows, norm_colname]
                    # append norm row as new df to norm_df
                    new_norm_df = pd.DataFrame([norm_row], columns=norm_colnames)
                    norm_df = norm_df.append(new_norm_df, ignore_index=True)
                    # also update colval_dict with new time/value and reset counter
                    colval_dict[col_no] = (new_time, new_value, 1)
                    
# completed iter thru all rows/cells in df_raw => flush out remaining readings in colval dict
for col_no, v in colval_dict.items():
    (new_time, new_value, new_delrows) = v       # unpack the value triplet 
    norm_colno = col_no            # use the column no instead of column name
    norm_stime = new_time          # inclusive of this time
    norm_etime = np.NaN            # exclusive of this time, which is unknown/missing
    norm_value = new_value         # value of sensor reading, treated as str
    norm_delrows = new_delrows
    norm_colname = raw_colnames[col_no]
    norm_row = [norm_colno, norm_stime, norm_etime, norm_value, norm_delrows, norm_colname]
    # append norm row as new df to norm_df
    new_norm_df = pd.DataFrame([norm_row], columns=norm_colnames)
    norm_df = norm_df.append(new_norm_df, ignore_index=True)

# finally sort and reindex the df
norm_df = norm_df.sort_values(['colno','stime'])
norm_df.index = range(len(norm_df))

# print for all to see...
print('\n', norm_df.head(5))
print('------')
print(norm_df.tail(5))

191999 99.999479166666664
    colno      stime      etime   value  delrows                  colname
0      1     0.7188   628.7188  Ground      627   Left Main Squat Switch
1      1   628.7188  3479.7188     Air     2851   Left Main Squat Switch
2      1  3479.7188        NaN  Ground      244   Left Main Squat Switch
3      2     0.1719   628.1719  Ground      627  Right Main Squat Switch
4      2   628.1719  3479.1719     Air     2851  Right Main Squat Switch
------
        colno     stime     etime value  delrows      colname
113056     56  625.8594  641.8594  Hold        1     A/T Hold
113057     56  641.8594       NaN     -      193     A/T Hold
113058     57    0.9375  609.9375     -      608  A/T Engaged
113059     57  609.9375  631.9375   ENG       22  A/T Engaged
113060     57  631.9375       NaN     -     3092  A/T Engaged


In [65]:
norm_df.head(20)

,colno,stime,etime,value,delrows,colname
0,1,0.7188,628.7188,Ground,627,Left Main Squat Switch
1,1,628.7188,3479.7188,Air,2851,Left Main Squat Switch
2,1,3479.7188,NaN,Ground,244,Left Main Squat Switch
3,2,0.1719,628.1719,Ground,627,Right Main Squat Switch
4,2,628.1719,3479.1719,Air,2851,Right Main Squat Switch
5,2,3479.1719,NaN,Ground,245,Right Main Squat Switch
6,3,0.0625,625.0625,Ground,624,Nose Squat Switch
7,3,625.0625,3483.0625,Air,2858,Nose Squat Switch
8,3,3483.0625,NaN,Ground,241,Nose Squat Switch
9,4,0.1250,8.1250,0,7,Calibrated airspeed


In [66]:
norm_df.tail(20)

,colno,stime,etime,value,delrows,colname
113041,53,3693.9688,3695.9688,18.75,1,Static Air Temp
113042,53,3695.9688,3697.9688,20.5,1,Static Air Temp
113043,53,3697.9688,3699.9688,22.5,1,Static Air Temp
113044,53,3699.9688,3701.9688,22.75,1,Static Air Temp
113045,53,3701.9688,3703.9688,21.75,1,Static Air Temp
113046,53,3703.9688,3705.9688,21,1,Static Air Temp
113047,53,3705.9688,3707.9688,20.75,1,Static Air Temp
113048,53,3707.9688,3709.9688,20.25,1,Static Air Temp
113049,53,3709.9688,3711.9688,20,1,Static Air Temp
113050,53,3711.9688,3713.9688,19.75,1,Static Air Temp


In [15]:
# Convert the raw df format to a 'normalize' format of (time, variable, value)

new_colnames = ['time', 'variable', 'value']
df_norm = pd.DataFrame(columns=new_colnames)
df_colnames = df_raw.columns
time_zero = df_raw.iloc[0,0]

for row_no, (row, series) in enumerate(df_raw.iloc[:, :].iterrows()):  # scan all variables (except time)
    print(100*row_no/len(df_raw), end='\r')       # log progress
    time = series[0]
    for col_no, value in enumerate(series[1:]):  # ignore time in first column
        if pd.notnull(value):
            new_row = [((time - time_zero), df_colnames[col_no+1], str(value))]
            df_norm = df_norm.append(pd.DataFrame(new_row, columns=new_colnames))

df_norm.index = range(len(df_norm))
print('\n', df_norm.head(5))
print('------')
print(df_norm.tail(5))


99.999479166666664466
      time                   variable               value
0  0.0000  Left wheel brake pressure              2485.0
1  0.0313      Vertical acceleration  1.0190000000000001
2  0.0469  Longitudinal acceleration              -0.017
3  0.0469           Gears in transit      Not in transit
4  0.0625          Nose Squat Switch              Ground
------
             time                  variable                  value
260886  3723.5625      Lateral acceleration  -0.037000000000000005
260887  3723.5625         FLT hyd syst fail                  Valid
260888  3723.5782         Elevator position                 -14.99
260889  3723.5782  Left thrust reverser cmd                Not cmd
260890  3723.5782                Master Red                    OFF


In [21]:
# Examine the frequency that readings are recorded for each variable

df_colnames = df_raw.columns
for i, colname in enumerate(df_colnames[1:]):
    df = df_norm[df_norm['variable'] == df_colnames[i+1]]
    #print(df.head())
    time0 = df.iloc[0,0]
    dur_list = []
    for (index, row) in df[1:].iterrows():
        #print(i, time0, index, row.tolist())
        time1 = row.iloc[0]
        col1 = row.iloc[1]
        val1 = row.iloc[2]
        dur = time1 - time0
        dur_list.append(dur)
        #print(time0, time1, dur, col1, val1)
        time0 = time1
    #print("----", dur_list)
    print([ (i, dur_list.count(i)) for i in set(dur_list) ], colname)


[(1.0, 3722)] Left Main Squat Switch
[(1.0, 3723)] Right Main Squat Switch
[(1.0, 3723)] Nose Squat Switch
[(1.0, 3723)] Calibrated airspeed
[(0.25, 14894)] Lateral acceleration
[(0.25, 14894)] Longitudinal acceleration
[(0.125, 29788)] Vertical acceleration
[(0.25, 14893)] Pitch angle
[(0.5, 7446)] Roll angle
[(1.0, 3723)] Magnetic heading
[(1.0, 3722)] True track angle
[(1.0, 3723)] Ground speed
[(0.5, 7446)] Rudder position
[(1.0, 3723)] Anti-skid fail
[(1.0, 3723)] Brake fail
[(0.25, 14894)] Elevator position
[(1.0, 3723)] Flaps 0 DEG
[(1.0, 3723)] Flaps 10 DEG
[(1.0, 3723)] Flaps 20 DEG
[(1.0, 3722)] Flaps 39 DEG
[(2.0, 1861)] FLT hyd syst fail
[(1.0, 3723)] Gears in transit
[(0.5, 7446)] Left aileron position
[(1.0, 3723)] Left EPR
[(4.0, 930)] Left fuel flow
[(1.0, 3723)] Left ground spoiler position
[(1.0, 3723)] Left inbd flight spoiler position
[(4.0, 930)] Left N1
[(2.0, 1861)] Left N2
[(1.0, 3723)] Left PLA
[(1.0, 3723)] Left thrust reverser cmd
[(1.0, 3723)] Left thrust re

In [27]:
# Collapse df_norm into df_temp temporal using (stime, etime) periods

new_colnames = ['stime', 'etime', 'variable', 'value']
df_temp = pd.DataFrame(columns=new_colnames)


df_colnames = df_raw.columns
for col_no, colname in enumerate(df_colnames[1:]):
    df = df_norm[df_norm['variable'] == colname]
    tim0 = df.iloc[0,0]
    val0 = df.iloc[0,2]
    #print(col_no, tim0, colname, val0)

    for (row_no, row) in df[1:50].iterrows():
        val1 = row.iloc[2]
        if val0 != val1:
            tim1 = row.iloc[0]
            new_row = [(tim0, tim1, colname, val0)]
            df_temp = df_temp.append(pd.DataFrame(new_row, columns=new_colnames))
            #print(col_no, row_no, tim0, tim1, val0, val1)
            
            tim0 = tim1   # start with this new time
            val0 = val1   # along with its new value
            
    # close final row with etime as NaN
    new_row = [(tim0, None, colname, val0)]
    df_temp = df_temp.append(pd.DataFrame(new_row, columns=new_colnames))
    #print(col_no, row_no, tim0, None, val0)

# print resulting df_temp temporal dataframe
df_temp.index = range(len(df_temp))
print('\n', df_temp.head(10))
print('------')
print(df_temp.tail(10))


      stime   etime                 variable   value
0   0.7188     NaN   Left Main Squat Switch  Ground
1   0.1719     NaN  Right Main Squat Switch  Ground
2   0.0625     NaN        Nose Squat Switch  Ground
3   0.1250   8.125      Calibrated airspeed     0.0
4   8.1250  10.125      Calibrated airspeed     7.0
5  10.1250  15.125      Calibrated airspeed     0.0
6  15.1250  16.125      Calibrated airspeed     7.0
7  16.1250  17.125      Calibrated airspeed     5.0
8  17.1250  26.125      Calibrated airspeed     0.0
9  26.1250  27.125      Calibrated airspeed     3.0
------
        stime     etime           variable  value
703   71.9688   81.9688    Static Air Temp   20.0
704   81.9688   93.9688    Static Air Temp  20.25
705   93.9688   95.9688    Static Air Temp   20.0
706   95.9688       NaN    Static Air Temp  20.25
707    0.5469       NaN  Yaw damper status    ENG
708    0.2500       NaN  Auto-pilot status      -
709    1.8594  625.8594           A/T Hold      -
710  625.8594  641.

In [28]:
df_temp

,stime,etime,variable,value
0,0.7188,NaN,Left Main Squat Switch,Ground
1,0.1719,NaN,Right Main Squat Switch,Ground
2,0.0625,NaN,Nose Squat Switch,Ground
3,0.1250,8.1250,Calibrated airspeed,0.0
4,8.1250,10.1250,Calibrated airspeed,7.0
5,10.1250,15.1250,Calibrated airspeed,0.0
6,15.1250,16.1250,Calibrated airspeed,7.0
7,16.1250,17.1250,Calibrated airspeed,5.0
8,17.1250,26.1250,Calibrated airspeed,0.0
9,26.1250,27.1250,Calibrated airspeed,3.0
